In [1]:
from datasets import load_dataset, load_metric, Audio,concatenate_datasets



In [2]:
import pandas as pd
import random

In [3]:
import torchaudio.transforms as tat
import torchaudio

In [4]:
import torch

In [5]:
data_files = {"train": "train.tsv", "validation": "validated.tsv"}
common_voice_train = load_dataset("cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn", data_files=data_files)
common_voice_train = concatenate_datasets([common_voice_train['train'],common_voice_train['validation']])
common_voice_test = load_dataset("cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn", split="test")

In [6]:
clips = load_dataset("cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips",split='train')

Resolving data files:   0%|          | 0/261022 [00:00<?, ?it/s]

In [7]:
clips = clips.cast_column("audio", Audio(sampling_rate=16000))

In [8]:
common_voice_train[0]

{'client_id': 'f91444c4bad593ac4060a768f54e4020c91c12b3036e4ae74005738cc7f21122e81f17d520b9525fcc13d684a0e88560f4f3fabd85b31eaed24a3e79ceb07e28',
 'path': 'common_voice_bn_31544430.mp3',
 'sentence': 'মজুমদার কুষ্টিয়ায় জন্মগ্রহণ করেন।',
 'up_votes': 2,
 'down_votes': 1,
 'age': None,
 'gender': None,
 'accents': None,
 'locale': 'bn',
 'segment': None}

In [9]:
common_voice_train = common_voice_train.train_test_split(train_size=0.09)['train']

In [10]:
common_voice_test = common_voice_test.train_test_split(test_size=0.1)['test']

In [11]:
common_voice_train.shape

(4702, 10)

In [12]:
common_voice_test.shape

(823, 10)

In [13]:
common_voice_train.features


{'client_id': Value(dtype='string', id=None),
 'path': Value(dtype='string', id=None),
 'sentence': Value(dtype='string', id=None),
 'up_votes': Value(dtype='int64', id=None),
 'down_votes': Value(dtype='int64', id=None),
 'age': Value(dtype='string', id=None),
 'gender': Value(dtype='string', id=None),
 'accents': Value(dtype='string', id=None),
 'locale': Value(dtype='string', id=None),
 'segment': Value(dtype='float64', id=None)}

In [14]:
common_voice_train = common_voice_train.remove_columns(["accents", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accents", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [15]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [16]:
show_random_elements(common_voice_train, num_examples=10)

,path,sentence
0,common_voice_bn_31680980.mp3,সেখানে চলচ্চিত্র নির্মাতা জর্জ লুকাস তাদের প্রতিবেশী ছিল।
1,common_voice_bn_31548153.mp3,এছাড়াও ওডিআই ইতিহাসে অভিষেকে সেরা বোলিং করেন তিনি।
2,common_voice_bn_31616799.mp3,এভাবেই তাদের পরিবারের জীবিকার ব্যবস্থা হতো।
3,common_voice_bn_31263843.mp3,"কেননা, ওল্ড ট্রাফোর্ডে খেলতে তিনি স্বাচ্ছন্দ্যবোধ করতেন না।"
4,common_voice_bn_31515315.mp3,এই সংগঠনটি ভারত সরকার এক সন্ত্রাসবাদী সংগঠন বলে ঘোষণা করেছে।
5,common_voice_bn_30992855.mp3,"এ দম্পতির দুই ছেলে, দুই মেয়ে।"
6,common_voice_bn_31552215.mp3,ব্যবসায়ের পাশাপাশি তিনি আফগান উইমেন্স নেটওয়ার্কের এক্সিকিউটিভ বোর্ড মেম্বার হিসেবেও দুই বছর যুক্ত ছিলেন।
7,common_voice_bn_31665310.mp3,মামলার শিরোনাম ছাড়া বাকি অভিযোগটি গোপন ছিল।
8,common_voice_bn_30994062.mp3,এগুলো ভিয়েতনামের শিশুশ্রমের অন্যতম কারণ।
9,common_voice_bn_31558290.mp3,এর মাধ্যমে তিনি একজন সমকামী হিসেবে আখ্যায়িত হওয়ার হাত থেকে নিজেকে রক্ষা করেন।


In [17]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\।]'

In [18]:
def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, "", batch["sentence"]).lower()
    return batch

In [19]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

Map:   0%|          | 0/4702 [00:00<?, ? examples/s]

Map:   0%|          | 0/823 [00:00<?, ? examples/s]

In [20]:
show_random_elements(common_voice_train)

,path,sentence
0,common_voice_bn_31574593.mp3,তিনি মালয়ালম লেখিকা এবং কণ্ঠশিল্পী
1,common_voice_bn_30991694.mp3,পোষা ও বন্য প্রাণীর উত্তম আবাসস্থল হিসেবে এটির ভূমিকা বিশাল ও ব্যাপক
2,common_voice_bn_31542753.mp3,এছাড়া ধারাবাহিকটিতে যুদ্ধে প্রচুর পরিমাণ মৃত্যুর চিত্র তুলে ধরা হয়েছে
3,common_voice_bn_31690658.mp3,এইসময় থেকেই উপদ্বীপের এই অঞ্চলের উপকূল বরাবর বেশ কিছু অস্থায়ী বা কিছু সময়ের জন্য স্থায়ী বাণিজ্যকেন্দ্র গড়ে উঠতে শুরু করে
4,common_voice_bn_31510009.mp3,বর্তমানে প্রতিষ্ঠানটির প্রধান কার্যালয় মহারাষ্ট্রের পুনে শহরে অবস্থিত
5,common_voice_bn_31697575.mp3,আইন পরিষদে নির্বাচিত হলো কলকাতার ইহুদীদের সর্বোচ্চ রাজনৈতিক কার্যালয়
6,common_voice_bn_31598992.mp3,আকাশ অন্য অনির্দিষ্ট জাতীয় টিভি পুরস্কার পেয়েছিলেন
7,common_voice_bn_31514886.mp3,তিনি বাংলাদেশ জাতীয়তাবাদী দলের নেতা সাবেক সাংসদ
8,common_voice_bn_31519953.mp3,তাঁর বাবা জেরি এবং মা ক্যাথলিন ছিলেন
9,common_voice_bn_31567369.mp3,ঐ খেলাটি ড্রয়ে পরিণত হয়েছিল


In [21]:
def extract_all_chars(batch):
    all_text = " ".join(batch["sentence"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

In [22]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/4702 [00:00<?, ? examples/s]

Map:   0%|          | 0/823 [00:00<?, ? examples/s]

In [23]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [24]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 "'": 1,
 '/': 2,
 'a': 3,
 'e': 4,
 'g': 5,
 'l': 6,
 'o': 7,
 'p': 8,
 'y': 9,
 'ঁ': 10,
 'ং': 11,
 'ঃ': 12,
 'অ': 13,
 'আ': 14,
 'ই': 15,
 'ঈ': 16,
 'উ': 17,
 'ঊ': 18,
 'ঋ': 19,
 'এ': 20,
 'ঐ': 21,
 'ও': 22,
 'ঔ': 23,
 'ক': 24,
 'খ': 25,
 'গ': 26,
 'ঘ': 27,
 'ঙ': 28,
 'চ': 29,
 'ছ': 30,
 'জ': 31,
 'ঝ': 32,
 'ঞ': 33,
 'ট': 34,
 'ঠ': 35,
 'ড': 36,
 'ঢ': 37,
 'ণ': 38,
 'ত': 39,
 'থ': 40,
 'দ': 41,
 'ধ': 42,
 'ন': 43,
 'প': 44,
 'ফ': 45,
 'ব': 46,
 'ভ': 47,
 'ম': 48,
 'য': 49,
 'র': 50,
 'ল': 51,
 'শ': 52,
 'ষ': 53,
 'স': 54,
 'হ': 55,
 '়': 56,
 'া': 57,
 'ি': 58,
 'ী': 59,
 'ু': 60,
 'ূ': 61,
 'ৃ': 62,
 'ে': 63,
 'ৈ': 64,
 'ো': 65,
 'ৌ': 66,
 '্': 67,
 'ৎ': 68,
 'ড়': 69,
 'য়': 70,
 '’': 71}

In [25]:
vocab_dict['|'] = vocab_dict[' ']
del vocab_dict[' ']

In [26]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

74

In [27]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [28]:
def add_path(record):
    home = '/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/'
    record['path'] = home+record['path']
    return record
    

In [29]:
common_voice_train = common_voice_train.map(add_path)
common_voice_test = common_voice_test.map(add_path)

Map:   0%|          | 0/4702 [00:00<?, ? examples/s]

Map:   0%|          | 0/823 [00:00<?, ? examples/s]

In [30]:
common_voice_train.shape

(4702, 2)

In [31]:
show_random_elements(common_voice_train)

,path,sentence
0,/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/common_voice_bn_31724482.mp3,তিনি বর্তমানে এটিএন নিউজের একজন সংবাদ উপস্থাপক
1,/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/common_voice_bn_31536308.mp3,তার ডাকনাম হচ্ছে নিনজা
2,/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/common_voice_bn_31551431.mp3,ফলশ্রুতিতে পাঁচজন খসড়া খেলোয়াড়ের অন্যতম হিসেবে শেষ মুহুর্তে খেলার সুযোগ লাভ করেন
3,/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/common_voice_bn_31639700.mp3,নিউ দিল্লী হোটেলস
4,/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/common_voice_bn_31663911.mp3,ঐ বছরই তিনি বার্লিন নগরে চলে আসেন আরো ভালোভাবে সংস্কৃত শিক্ষালাভের জন্য
5,/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/common_voice_bn_31630686.mp3,তিনি কয়েকটি অপারেশনে যথেষ্ট সাহসিকতা প্রদর্শন করেন
6,/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/common_voice_bn_31011596.mp3,বহু ইউরোপীয়দের বাসস্থান হওয়ার জন্য এই লোকালয়টি পূর্বে হোয়াইট টাউন নামে পরিচিত ছিল
7,/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/common_voice_bn_31507166.mp3,তিনি তার বাকি জীবন তার অন্য ছেলের সাথে কাটান
8,/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/common_voice_bn_31336837.mp3,রামগতি দশ বৎসর বয়স পর্যন্ত পড়াশোনা করেন গ্রামের পাঠশালায়
9,/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/common_voice_bn_31703092.mp3,চলচ্চিত্রটি ছিল আমজাদ খান পরিচালিত প্রথম চলচ্চিত্র


In [32]:
class W2v2Dataset(torch.utils.data.Dataset):
    def __init__(self, df, processor):
        self.df = df
        self.pathes = df['path']
        self.sentences = df['sentence']
        self.resampler = tat.Resample(32000, 16000)
        self.processor = processor

    def __getitem__(self, idx):
        apath = self.pathes[idx]
        waveform, sample_rate = torchaudio.load(apath, format="wav")
        waveform = self.resampler(waveform)
        batch = dict()
        y = self.processor(waveform.reshape(-1), sampling_rate=16000).input_values[0] 
        batch["input_values"] = y
        with self.processor.as_target_processor():
            batch["labels"] = self.processor(self.sentences[idx]).input_ids       
        
        return batch

    def __len__(self):
        return len(self.df)

In [33]:
from transformers import Wav2Vec2CTCTokenizer

/home/neeraj/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/neeraj/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [34]:
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


In [35]:
from transformers import Wav2Vec2FeatureExtractor


In [36]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [37]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [38]:
common_voice_train[0]['path']

'/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/common_voice_bn_30698857.mp3'

In [39]:
common_voice_train[0]

{'path': '/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/common_voice_bn_30698857.mp3',
 'sentence': 'তবে হুমায়ুনের সাহায্যের আহবান থাকা সত্ত্বেও কামরান তাকে কোনও সাহায্যের প্রস্তাব দেয়নি'}

In [40]:
common_voice_train = W2v2Dataset(common_voice_train,processor=processor)
common_voice_test = W2v2Dataset(common_voice_test, processor=processor)

In [41]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

In [42]:
@dataclass
class DataCollatorCTCWithPadding:

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:

        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [43]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [44]:
data_collator

DataCollatorCTCWithPadding(processor=Wav2Vec2Processor:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

- tokenizer: Wav2Vec2CTCTokenizer(name_or_path='./', vocab_size=74, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	72: AddedToken("[UNK]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	73: AddedToken("[PAD]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	74: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	75: AddedToken("</s>", rstrip=

In [45]:
wer_metric = load_metric("wer")

/tmp/ipykernel_119096/24688115.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [46]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [47]:
from transformers import Wav2Vec2ForCTC

/home/neeraj/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/neeraj/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [48]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m", 
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
model.save_pretrained("my-wav2vec2-model-bengali-version-2")

In [50]:
# model = Wav2Vec2ForCTC.from_pretrained('my-wav2vec2-model')

In [51]:
len(processor.tokenizer)

76

In [52]:
model.freeze_feature_encoder()

In [53]:
model

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projec

In [54]:
from transformers import TrainingArguments

In [65]:
training_args = TrainingArguments(
  output_dir='xlsr-wav2vec2-for-bengali',
  group_by_length=True,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=4,
  evaluation_strategy="steps",
  num_train_epochs=8,
  gradient_checkpointing=True,
  fp16=False,
  save_steps=400,
  eval_steps=400,
  logging_steps=50,
  learning_rate=3e-4,
  warmup_steps=100,
  save_total_limit=2,
  push_to_hub=False,
)

In [66]:
from transformers import Trainer

In [67]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

In [68]:
import numpy as np

In [69]:
trainer.train()

Step,Training Loss,Validation Loss,Wer
400,1.721000,1.255541,0.871246
800,0.318200,0.474099,0.450248


Checkpoint destination directory xlsr-wav2vec2-for-bengali/checkpoint-400 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/home/neeraj/anaconda3/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/neeraj/anaconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.w

TrainOutput(global_step=1176, training_loss=1.561104806102052, metrics={'train_runtime': 82276.5922, 'train_samples_per_second': 0.457, 'train_steps_per_second': 0.014, 'total_flos': 6.977006478167455e+18, 'train_loss': 1.561104806102052, 'epoch': 8.0})

In [70]:
trainer

In [71]:
trainer.save_model('Saved_trainer_bengali_version-2/')

In [72]:
trainer

In [103]:
# from huggingface_hub import notebook_login


In [104]:
# notebook_login()

In [89]:
# repo = 'xlsr-wav2vec2-for-assamese'

In [73]:

common_voice_test_transcription = load_dataset("cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn", split="test")

In [74]:
common_voice_test_transcription

Dataset({
    features: ['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accents', 'locale', 'segment'],
    num_rows: 8226
})

In [75]:
common_voice_test_transcription = common_voice_test_transcription.remove_columns(["accents", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [76]:
common_voice_test_transcription

Dataset({
    features: ['path', 'sentence'],
    num_rows: 8226
})

In [77]:
common_voice_test_transcription = common_voice_test_transcription.train_test_split(train_size = 0.05)['train']

In [78]:
common_voice_test_transcription

Dataset({
    features: ['path', 'sentence'],
    num_rows: 411
})

In [79]:
common_voice_test_transcription = common_voice_test_transcription.map(remove_special_characters)

Map:   0%|          | 0/411 [00:00<?, ? examples/s]

In [80]:
show_random_elements(common_voice_test_transcription)

,path,sentence
0,common_voice_bn_31553376.mp3,পাশাপাশি অনেক ধর্মযাজক ও প্রজাতান্ত্রিক অভ্যুত্থানের মাধ্যমে রাজনৈতিক ইতিহাস গড়ে উঠে
1,common_voice_bn_31569102.mp3,পৃষ্ঠীয় ভাগ উদরীয় ভাগ অপেক্ষা অধিক উত্তল
2,common_voice_bn_31532231.mp3,ঠোঁটে মাছ বা অন্যান্য খাবারের অস্তিত্ব টের পেলেই এরা সাথে সাথে ঠোঁট বন্ধ করে ফেলে
3,common_voice_bn_31572188.mp3,তিনি ছিলেন হযরত দাউদএর পুত্র
4,common_voice_bn_31649901.mp3,জনগণের পক্ষে এ সভায় দাবী উত্থাপন করেন সৈয়দ সুলতান আলী
5,common_voice_bn_30992467.mp3,টিম কুক যুক্তরাষ্ট্রের এয়লাবামায় বেড়ে ওঠেন
6,common_voice_bn_31751017.mp3,এটি একটি অত্যন্ত উর্বর ভূমি
7,common_voice_bn_31559977.mp3,ভ্যান একবার জলে নামলে অভিনেতাদের কাছে চ্যালেঞ্জ হ'ল আতঙ্ক এড়ানো
8,common_voice_bn_31512168.mp3,তারপরে শিউলি আজাদ তার শহীদ স্বামীর স্বপ্ন বাস্তবায়নের জন্য কাজ করছেন
9,common_voice_bn_31533100.mp3,বর্তমানে এ বিশ্ববিদ্যালয়ের উপাচার্য হিসেবে দায়িত্ব পালন করছেন প্রফেসর ডাক্তার মোহাম্মদ নজরুল ইসলাম


In [81]:
common_voice_test_transcription = common_voice_test_transcription.map(add_path)

Map:   0%|          | 0/411 [00:00<?, ? examples/s]

In [83]:
show_random_elements(common_voice_test_transcription)

,path,sentence
0,/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/common_voice_bn_31523662.mp3,তিনি স্কুল থেকে তিনি রাজনৈতিক জীবন শুরু করেন
1,/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/common_voice_bn_31572188.mp3,তিনি ছিলেন হযরত দাউদএর পুত্র
2,/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/common_voice_bn_31666486.mp3,যিনি বাংলাদেশের মুক্তিযুদ্ধে শহীদ হন
3,/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/common_voice_bn_31258205.mp3,পিএইচপি সামাজিক ও রাজনৈতিক বিষয়ে হিন্দু সম্প্রদায়কে প্রতিনিধিত্ব করে
4,/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/common_voice_bn_31258202.mp3,তিনি দ্বিতীয়বার বলটি মারার পরেও বলটি পুনরায় লেকে পতিত হয়
5,/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/common_voice_bn_31610915.mp3,তিনি ফার্সি ভাষা রপ্ত করেছিলেন
6,/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/common_voice_bn_31552855.mp3,তিনি ঢাকার হোটেল সুন্দরবন জব্বার টাওয়ার ও খুলনার হোটেল পার্কের মালিক ছিলেন
7,/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/common_voice_bn_31605782.mp3,হোটেল এ একটি শপিং মল ও আছে
8,/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/common_voice_bn_31572766.mp3,এটি ওশেনিয়া অঞ্চলে অবস্থিত
9,/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/common_voice_bn_31658713.mp3,আসল হো ম্যান টিন আজকের হো ম্যান টিন থেকে বেশ আলাদা ছিল


In [84]:
import os
import librosa

In [85]:
def add_audio(x):
    audio_data,sr = librosa.load(x['path'],sr=16000)
    return {'audio': {"array":audio_data,'sampling_rate':sr}}

In [86]:
common_voice_inference = common_voice_test_transcription.map(add_audio)

Map:   0%|          | 0/411 [00:00<?, ? examples/s]

In [87]:
common_voice_inference

Dataset({
    features: ['path', 'sentence', 'audio'],
    num_rows: 411
})

In [88]:
common_voice_inference['audio'][8]

{'array': [-1.1641532182693481e-10,
  -2.6193447411060333e-10,
  -1.7462298274040222e-10,
  -2.1827872842550278e-10,
  -5.820766091346741e-11,
  -1.4551915228366852e-10,
  -5.093170329928398e-11,
  -2.6193447411060333e-10,
  -1.4551915228366852e-10,
  -4.3655745685100555e-11,
  8.731149137020111e-11,
  5.820766091346741e-11,
  1.4551915228366852e-10,
  2.6193447411060333e-10,
  2.6193447411060333e-10,
  -1.6007106751203537e-10,
  -4.3655745685100555e-11,
  -7.275957614183426e-11,
  -3.7834979593753815e-10,
  -4.3655745685100555e-10,
  1.3096723705530167e-10,
  3.2014213502407074e-10,
  -4.3655745685100555e-10,
  4.3655745685100555e-11,
  -2.9103830456733704e-10,
  3.92901711165905e-10,
  -2.473825588822365e-10,
  7.057678885757923e-10,
  -2.9103830456733704e-11,
  -1.7680577002465725e-09,
  -1.6880221664905548e-09,
  -7.566995918750763e-10,
  8.294591680169106e-10,
  5.675246939063072e-10,
  3.637978807091713e-09,
  1.5279510989785194e-09,
  -1.6589183360338211e-09,
  -1.58615875989198

In [89]:
import IPython.display as ipd

In [90]:
ipd.Audio(data=common_voice_inference[0]['audio']['array'], autoplay=True, rate=16000)


In [91]:
common_voice_inference[0]['sentence']

'সেখান থেকে উত্তর দিকে প্রবাহিত হয়ে মৌলভীবাজার জেলার কমলগঞ্জ উপজেলা দিয়ে বাংলাদেশে প্রবেশ করেছে'

In [92]:
def prepare_dataset(batch):
    audio = batch['audio']
    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch    

In [93]:
common_voice_inference = common_voice_inference.map(prepare_dataset)

Map:   0%|          | 0/411 [00:00<?, ? examples/s]

/home/neeraj/anaconda3/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [94]:
common_voice_inference['input_length'][0]

115776

In [95]:
common_voice_inference.features

{'path': Value(dtype='string', id=None),
 'sentence': Value(dtype='string', id=None),
 'audio': {'array': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None),
  'sampling_rate': Value(dtype='int64', id=None)},
 'input_values': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None),
 'input_length': Value(dtype='int64', id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [96]:
common_voice_inference[8]

{'path': '/home/neeraj/Documents/AsrInJupyter/cv-corpus-10.0-2022-07-04-bn/cv-corpus-10.0-2022-07-04/bn/clips/common_voice_bn_31575149.mp3',
 'sentence': 'বিকানের শহরটি জেলা সদর',
 'audio': {'array': [-1.1641532182693481e-10,
   -2.6193447411060333e-10,
   -1.7462298274040222e-10,
   -2.1827872842550278e-10,
   -5.820766091346741e-11,
   -1.4551915228366852e-10,
   -5.093170329928398e-11,
   -2.6193447411060333e-10,
   -1.4551915228366852e-10,
   -4.3655745685100555e-11,
   8.731149137020111e-11,
   5.820766091346741e-11,
   1.4551915228366852e-10,
   2.6193447411060333e-10,
   2.6193447411060333e-10,
   -1.6007106751203537e-10,
   -4.3655745685100555e-11,
   -7.275957614183426e-11,
   -3.7834979593753815e-10,
   -4.3655745685100555e-10,
   1.3096723705530167e-10,
   3.2014213502407074e-10,
   -4.3655745685100555e-10,
   4.3655745685100555e-11,
   -2.9103830456733704e-10,
   3.92901711165905e-10,
   -2.473825588822365e-10,
   7.057678885757923e-10,
   -2.9103830456733704e-11,
   -1.768

In [97]:
input_dict = processor(common_voice_inference[8]["input_values"], return_tensors="pt", padding=True,sampling_rate=16000)


In [98]:
logits = model(input_dict.input_values).logits


/home/neeraj/anaconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


In [99]:
pred_ids = torch.argmax(logits, dim=-1)[0]

In [100]:
print(processor.decode(pred_ids))

বিকেনের সমরটি দেলা সদর


In [101]:
common_voice_inference['sentence'][8]

'বিকানের শহরটি জেলা সদর'

In [102]:
def decoded_voice(encoded):
    input_dict = processor(encoded, return_tensors="pt", padding=True,sampling_rate=16000)
    logits = model(input_dict.input_values).logits
    pred_ids = torch.argmax(logits, dim=-1)[0]
    return processor.decode(pred_ids)

In [103]:
decoded_voice(common_voice_inference[1]["input_values"])

'এক বছর পর তিনি আমেরিকা ফিড়েগিয়ো হারবার্ডের শিক্ষাকতা চালিয়ে যান'

In [104]:
common_voice_inference['sentence'][1]

'একবছর পর তিনি আমেরিকা ফিরে গিয়ে হার্ভার্ডে শিক্ষকতা চালিয়ে যান'

In [105]:
for i in range(5):
    print(f"Original value = {common_voice_inference[i]['sentence'].lower()}")
    print(f"Decoded value' = {decoded_voice(common_voice_inference[i]['input_values'])}")
    print('-------------------------')

Original value = সেখান থেকে উত্তর দিকে প্রবাহিত হয়ে মৌলভীবাজার জেলার কমলগঞ্জ উপজেলা দিয়ে বাংলাদেশে প্রবেশ করেছে
Decoded value' = সেখান থেকে উত্তর দিকে প্রবাহিত হয়ে মলবি বাজর জেলার কমনগঞ্জ উপজেলাদীয় বাংলাদেশের পবেশ করেছে
-------------------------
Original value = একবছর পর তিনি আমেরিকা ফিরে গিয়ে হার্ভার্ডে শিক্ষকতা চালিয়ে যান
Decoded value' = এক বছর পর তিনি আমেরিকা ফিরে গিয়ো হারবার্ডের শিক্ষাকতা চালিয়ে যান
-------------------------
Original value = এছাড়াও তিনি দুইবারের ওয়ার্ল্ড চ্যাম্পিয়ন
Decoded value' = এছাড়াও তিনি দুইবারের ওয়ার্ড চ্যান্টন
-------------------------
Original value = তার ফুফাতো বোন বরেন্দ্র বিশ্ববিদ্যালয়ে পড়ে
Decoded value' = থার পোপাত পণ পরেন্ট্র বিশবিতযালয়ে পড়ে
-------------------------
Original value = এর অধিকাংশই ফ্রি সফটওয়্যার লাইসেন্সের অধিনে প্রকাশিত
Decoded value' = এর অধিকাংশ ভরিসোপট্রযালাইসেতে অধিনিপকেশিত
-------------------------


In [106]:
import csv

In [109]:
with open("decoded_voice_bengali_v2.csv", "w", newline="") as f:
    writer = csv.writer(f)

    
    writer.writerow(["Original value", "Decoded value"])

    for i in range(len(common_voice_inference)):
        original_value = common_voice_inference[i]["sentence"].lower()
        decoded_value = decoded_voice(common_voice_inference[i]["input_values"])

        writer.writerow([original_value, decoded_value])

        
f.close()

In [110]:
import pandas as pd

In [111]:
df = pd.read_csv('decoded_voice_bengali_v2.csv')

In [112]:
df

,Original value,Decoded value
0,সেখান থেকে উত্তর দিকে প্রবাহিত হয়ে মৌলভীবাজার...,সেখান থেকে উত্তর দিকে প্রবাহিত হয়ে মলবি বা জর...
1,একবছর পর তিনি আমেরিকা ফিরে গিয়ে হার্ভার্ডে শি...,এক বছর পর তিনি আমেরিকা ফিরে গিয়ো হারবার্ডের ...
2,এছাড়াও তিনি দুইবারের ওয়ার্ল্ড চ্যাম্পিয়ন,এছাড়াও তিনি দুইবারের ওয়ার্ড কাম্পিয়ন
3,তার ফুফাতো বোন বরেন্দ্র বিশ্ববিদ্যালয়ে পড়ে,থার পোপাত পোণ পরেন্ট্র বিশ্ববিতযালয়ে পড়ে
4,এর অধিকাংশই ফ্রি সফটওয়্যার লাইসেন্সের অধিনে প...,এর অধিকাংশে ভ্রিসোপট্া লাইসেনসে অধিনিপরকাশিত
...,...,...
406,সরকার জেএমবিসহ চারটি দল নিষিদ্ধ করে,সরকার যে এবি সহচারটি দল লিষিদ্ধ করে
407,এদেরকে প্রায়ই পুরাণে শক্তি ও ধার্মিকতায় দেবত...,এদেরে প্রায়ই পুরানের সতিগ্ ধর্মধার্মীকতা ়িদ্...
408,অতঃপর এটি ভেঙ্গে দুটি নতুন সংস্থা গঠিত হয়েছে ...,অত্যপর এটি ভেঙ্গে দুইটি নতুন সংস্থা গৃত হয়েছে...
409,তার প্রথম লক্ষ্য ছিল নতুন সুলতানের রাজত্ব সুরক...,তার প্রথম লক্য ছিল নতুম সৌতানেই রাযিত্য শিরপর্...


In [113]:
from datasets import load_metric

In [114]:
wer = load_metric('wer')

In [115]:
cer = load_metric('cer')

In [116]:
wer_score = wer.compute(predictions=df['Decoded value'], references=df['Original value'])

In [117]:
wer_score

0.451063829787234

In [118]:
cer_score = cer.compute(predictions=df['Decoded value'], references=df['Original value'])

In [119]:
cer_score

0.12666338097585017

In [120]:
from bnunicodenormalizer import Normalizer 
bnorm  = Normalizer()

In [121]:
def normalize(sen):
    sen = str(sen)
    _words = [bnorm(word)['normalized']  for word in sen.split()]
    return " ".join([word for word in _words if word is not None])

In [122]:
df['Decoded value']= df['Decoded value'].apply(lambda x:normalize(x))
df['Original value']= df['Original value'].apply(lambda x:normalize(x))

In [123]:
wer_score = wer.compute(predictions=df['Decoded value'], references=df['Original value'])
cer_score = cer.compute(predictions=df['Decoded value'], references=df['Original value'])

In [124]:
wer_score

0.44627659574468087

In [125]:
cer_score

0.1268853143931481